In [1]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import os
import pandas as pd
import numpy as np
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential

from keras import layers
from keras import Model


from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from keras import optimizers
from keras import backend as K

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau



from sklearn.metrics import classification_report
from keras.optimizers import SGD


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [3]:
train_path = 'train/'
valid_path = 'test/'

# train_path = 'sources/train'
# valid_path = 'sources/test'

# num_train_samples = 8015 # len(df_train)
# num_val_samples = 2000    # len(df_val)


num_train_samples = 932 # len(df_train)
num_val_samples = 170  # len(df_val)
train_batch_size = 10
val_batch_size = 10

image_size = 224
# image_size = 300

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)




train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (image_size,image_size),
                                                 batch_size = train_batch_size)


test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (image_size,image_size),
                                            batch_size = val_batch_size)

Found 932 images belonging to 2 classes.
Found 170 images belonging to 2 classes.


In [4]:
pre_trained_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

W1113 23:28:10.516877 15544 deprecation_wrapper.py:119] From C:\Users\S\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1113 23:28:10.517875 15544 deprecation_wrapper.py:119] From C:\Users\S\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1113 23:28:10.969386 15544 deprecation_wrapper.py:119] From C:\Users\S\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1113 23:28:11.203440 15544 deprecation_wrapper.py:119] From C:\Users\S\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1113 23:28:12.735776 15544 deprecation_wrapper.py:119] From C:\Users\S\Anaconda3\lib\site-pack

In [6]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output


x = layers.GlobalMaxPooling2D()(last_output)

x = layers.Dense(512, activation='relu')(x)

x = layers.BatchNormalization(axis=-1)(x)


x = layers.Dropout(0.5)(x)

x = layers.Dense(2, activation='softmax')(x)



model = Model(pre_trained_model.input, x)
# optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

last layer output shape: (None, 7, 7, 512)


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
print(test_set.class_indices)

{'mama': 0, 'nonmama': 1}


In [9]:
class_weights={
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 1.0, # df
    4: 5.0, # mel
    5: 1.0, # nv
    6: 1.0, # vasc
}

In [10]:
history = model.fit_generator(training_set,
                    steps_per_epoch = 100,
                    epochs = 5,
                    validation_data = test_set,
                    validation_steps = val_steps)

W1113 23:29:01.804362 15544 deprecation.py:323] From C:\Users\S\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
100/100 [==============================] - 179s 2s/step - loss: 0.6056 - acc: 0.7320 - val_loss: 0.3020 - val_acc: 0.8529
Epoch 2/5
100/100 [==============================] - 73s 734ms/step - loss: 0.4130 - acc: 0.8360 - val_loss: 0.2261 - val_acc: 0.9059
Epoch 3/5
100/100 [==============================] - 76s 757ms/step - loss: 0.4176 - acc: 0.8250 - val_loss: 1.5097 - val_acc: 0.7059
Epoch 4/5
100/100 [==============================] - 76s 762ms/step - loss: 0.3444 - acc: 0.8560 - val_loss: 0.9199 - val_acc: 0.7118
Epoch 5/5
100/100 [==============================] - 78s 777ms/step - loss: 0.3090 - acc: 0.8810 - val_loss: 0.2832 - val_acc: 0.8941


In [11]:
model.save('model_VGG16.h5')

In [12]:
test_batches = test_datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)


predictions = model.predict_generator(test_batches, steps=170, verbose=1)


Found 170 images belonging to 2 classes.
170/170 [==============================] - 15s 89ms/step


In [13]:
y_pred = np.argmax(predictions, axis=-1)
len(y_pred)

170

In [14]:
sum(y_pred==test_batches.classes)

152

In [15]:
len(test_batches.classes)

170

In [16]:
sum(y_pred==test_batches.classes)/len(y_pred)

0.8941176470588236

In [17]:
test_labels = test_batches.classes

cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

cm

array([[ 33,  17],
       [  1, 119]], dtype=int64)

In [ ]:
target_names = ['mama','nonmama']
print(classification_report(test_batches.classes, y_pred, target_names=target_names))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()